In [18]:
import import_ipynb
# import plots_help

In [ ]:
import cv2
import torch
from torch import nn
from models import LinkNet34
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image, ImageFilter
import time
import sys
import ssl

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LinkNet34()
# model.load_state_dict(torch.load('linknet.pth'))
model.load_state_dict(torch.load('linknet.pth', map_location=lambda storage, loc: storage))
model.eval()
model.to(device)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LinkNet34()
# model.load_state_dict(torch.load('linknet.pth'))
model.load_state_dict(torch.load('linknet.pth', map_location=lambda storage, loc: storage))
model.eval()
model.to(device)

In [21]:

import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

In [25]:
##################################################
########################

In [26]:
import cv2
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

import random



In [27]:

#calculates frame rate of a video

In [28]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
import random
import pandas as pd

def frame_ps_video(DEFAULT_VIDEO):
    video = cv2.VideoCapture(DEFAULT_VIDEO)


    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

    if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)

        print( "Frames per second : {0}".format(fps))
        video.release()
        return fps
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print( "Frames per second  : {0}".format(fps))
        video.release()
        return fps

eye_remove = True
#can change to true/false

CASCADE_PATH = "haarcascade_frontalface_default.xml"

#DEFAULT_VIDEO = "C:\\Users\\Jayant\\Desktop\\ML_mini_prj\\subject27\\vid.avi"  #true val-89bpm
DEFAULT_VIDEO = "/Users/tgit/Desktop/mahika/sem6/ml/ML_Proj/rppgVitalMeasurments/UBFC_DATASET/DATASET_2/subject42/vid.avi" #true val-110 bpm 


MIN_FACE_SIZE = 100

WIDTH_FRACTION = 0.8 # Fraction of bounding box width to include in ROI
HEIGHT_FRACTION = 1

FPS = frame_ps_video(DEFAULT_VIDEO)
jojo=8   #window_time_sec initialize

if FPS>=20 and FPS<25:
    jojo=10
elif FPS>=25 and FPS<=32 :
    jojo=30
elif FPS>32:
    jojo=20

WINDOW_TIME_SEC = jojo

WINDOW_SIZE = int(np.ceil(WINDOW_TIME_SEC * FPS))
MIN_HR_BPM = 50.0
MAX_HR_BMP = 240.0
MAX_HR_CHANGE = 12.0
SEC_PER_MIN = 60

EYE_LOWER_FRAC = 0.35
EYE_UPPER_FRAC = 0.5





Frames per second  : 29.948009


In [29]:
def roi_util(image, faceBox): 

    widthFrac = WIDTH_FRACTION
    heigtFrac = HEIGHT_FRACTION

    # Adjust bounding box
    (x, y, w, h) = faceBox
    widthOffset = int((1 - widthFrac) * w / 2)
    heightOffset = int((1 - heigtFrac) * h / 2)
    faceBoxAdjusted = (x + widthOffset, y + heightOffset,int(widthFrac * w), int(heigtFrac * h))
    (x, y, w, h) = faceBoxAdjusted
    backgrndMask = np.full(image.shape, True, dtype=bool)
    backgrndMask[y:y+h, x:x+w, :] = False 

    if eye_remove:
        backgrndMask[int(y + h * EYE_LOWER_FRAC) :int( y + h * EYE_UPPER_FRAC), :] = True


    roi = np.ma.array(image, mask=backgrndMask) # Masked array
    return roi


In [30]:

# Sum of square differences between x1, x2, y1, y2 points for each ROI


def distance(roi1, roi2):
    return sum((roi1[i] - roi2[i])**2 for i in range(len(roi1)))

def roi_extract(frame, faceCascade, previousFaceBox):
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(grey, scaleFactor=1.1,minNeighbors=5, minSize=(MIN_FACE_SIZE, MIN_FACE_SIZE), flags=cv2.CASCADE_SCALE_IMAGE)

    roi = None
    faceBox = None

    # If no face detected, use ROI from previous frame
    if len(faces) == 0:
        faceBox = previousFaceBox

    # if many faces detected, use one closest to that from previous frame
    #standard way
    elif len(faces) > 1:
        if previousFaceBox is not None:
            # Find closest
            minDist = float("inf")
            for face in faces:
                if distance(previousFaceBox, face) < minDist:
                    faceBox = face
        else:
            # Chooses largest box by area 
            maxArea = 0
            for face in faces:
                if (face[2] * face[3]) > maxArea:
                    faceBox = face

    # If only one face dectected
    else:
        faceBox = faces[0]

        # Show rectangle
        #(x, y, w, h) = faceBox
        #cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)
    if faceBox is not None:
        roi = roi_util(frame, faceBox)
    return faceBox, roi


In [31]:

def extract_heartrate(window, lastHR):
    # Normalize across the window to have zero-mean and unit variance
    mean = np.mean(window, axis=0)
    std = np.std(window, axis=0)
    normalized = (window - mean) / std

    # decompose into three source signals using ICA
    ica = FastICA(tol=0.01,n_components=3)
    srcSig = ica.fit_transform(normalized)

    # pca = PCA()
    # srcSig = pca.fit_transform(normalized)
    #uncommment to decompose using PCA

    # Find power spectrum
    powerSpec = np.abs(np.fft.fft(srcSig, axis=0))**2
    freqs = np.fft.fftfreq(WINDOW_SIZE, 1.0 / FPS)

    # Find heart rate
    maxPwrSrc = np.max(powerSpec, axis=1)
    validIdx = np.where((freqs >= MIN_HR_BPM / SEC_PER_MIN) & (freqs <= MAX_HR_BMP / SEC_PER_MIN))
    validPwr = maxPwrSrc[validIdx]
    validFreqs = freqs[validIdx]
    maxPwrIdx = np.argmax(validPwr)
    hr = validFreqs[maxPwrIdx]
#     print (hr) #printing heart beats/sec as time window slides

    # plot
    print("Recorded: "+str(hr))
    return hr


In [32]:

# if __name__ == "__main__":


video = cv2.VideoCapture(DEFAULT_VIDEO)
faceCascade = cv2.CascadeClassifier(CASCADE_PATH)

colorSig = [] # Will store the average RGB color values in each frame's ROI
heartRates = [] # Will store the heart rate calculated every 1 second
previousFaceBox = None
while True:
    # Capture frame-by-frame
    ret, frame = video.read()
    if not ret:
        break

#     previousFaceBox, roi = roi_extract(frame, faceCascade, previousFaceBox)
#     print(roi.shape)
    
    #face segmenation - cnn pre-trained resnet
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    img_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    shape = frame.shape[0:2]
    frame_ = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_,(256,256), cv2.INTER_LINEAR )
    
    a = img_transform(Image.fromarray(frame_))
    a = a.unsqueeze(0)
    imgs = Variable(a.to(dtype=torch.float, device=device))
    pred = model(imgs)

    pred = torch.nn.functional.interpolate(pred, size=[shape[0], shape[1]])
    mask = pred.data.cpu().numpy()
    mask = mask.squeeze()

    mask = mask > 0.8
    rgba = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    ind = frame[np.where(mask != 0)]
    non_mask_ind = np.where(mask == 0) #places outside face mask
    rgba[non_mask_ind] = [0,0,0,0]
    roi = rgba

    
    #print(previousFaceBox) #eg--[275 181 160 160]
#     cc=1
#     if(cc==1):
#         print(roi)
#         cc+=1

    if (roi is not None) and (np.size(roi) > 0):
        colorChannels = roi.reshape(-1, roi.shape[-1])
        avgColor = colorChannels.mean(axis=0)
        colorSig.append(avgColor)

    # Calculate heart rate every one second (once have 10sec or 15 sec or 20 sec of data depends on jojo)
    if (len(colorSig) >= WINDOW_SIZE) and (len(colorSig) % np.ceil(FPS) == 0):
        windowStart = len(colorSig) - WINDOW_SIZE
        window = colorSig[windowStart : windowStart + WINDOW_SIZE]
        lastHR = heartRates[-1] if len(heartRates) > 0 else None
        heartRates.append(extract_heartrate(window, lastHR))

#     if np.ma.is_masked(roi):
#         roi = np.where(roi.mask == True, 0, roi)
    
    cv2.imshow('Image',roi)
    cv2.waitKey(1)

print()
print( "just a theoretical mean ",np.mean(heartRates)*60)
print()
print("Instances captured --",len(heartRates))
 
pd.DataFrame(heartRates).to_csv("result_bps.csv")
video.release()
cv2.destroyAllWindows()

calculating hr...
extracting heartrate...
1.5990038175750834
Recorded: 1.5990038175750834
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculatin

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978
calculating hr...
extracting heartrate...
1.6323163971078978
Recorded: 1.6323163971078978

just a theoretical mean  97.99023394883207

Instances captured -- 39
